In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

print("""
# #################################################################################################
#  Full ANN Code Along - Regression Part One(1) - Feature Engineering
# #################################################################################################
""")


# #################################################################################################
#  Full ANN Code Along - Regression Part One(1) - Feature Engineering
# #################################################################################################



In [2]:
df = pd.read_csv('../../../notebooks/Data/NYCTaxiFares.csv')
print(df.head())
print(df['fare_amount'].describe())

           pickup_datetime  fare_amount  fare_class  pickup_longitude  \
0  2010-04-19 08:17:56 UTC          6.5           0        -73.992365   
1  2010-04-17 15:43:53 UTC          6.9           0        -73.990078   
2  2010-04-17 11:23:26 UTC         10.1           1        -73.994149   
3  2010-04-11 21:25:03 UTC          8.9           0        -73.990485   
4  2010-04-17 02:19:01 UTC         19.7           1        -73.990976   

   pickup_latitude  dropoff_longitude  dropoff_latitude  passenger_count  
0        40.730521         -73.975499         40.744746                1  
1        40.740558         -73.974232         40.744114                1  
2        40.751118         -73.960064         40.766235                2  
3        40.756422         -73.971205         40.748192                1  
4        40.734202         -73.905956         40.743115                1  
count    120000.000000
mean         10.040326
std           7.500134
min           2.500000
25%           5.700

In [3]:
def haversine_distance(df, src_lat, src_long, tar_lat, tar_long):
    """
    Calculates the haversine distance between 2 sets of GPS coordinates in df
    """
    r = 6371  # average radius of Earth in kilometers

    phi1 = np.radians(df[src_lat])
    phi2 = np.radians(df[tar_lat])

    delta_phi = phi2 - phi1
    delta_lambda = np.radians(df[tar_long] - df[src_long])

    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = (r * c)  # in kilometers
    return d

In [4]:
# FEATURE ENGINEERING - Taking a existing features already have, create new more useful features
# than original features.

# Engineering distance between two points with co-ordinates.
df['dist_km'] = haversine_distance(df, 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
                                   'dropoff_longitude')
print(df['dist_km'].head())

print(df.info())

0    2.126312
1    1.392307
2    3.326763
3    1.864129
4    7.231321
Name: dist_km, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   pickup_datetime    120000 non-null  object 
 1   fare_amount        120000 non-null  float64
 2   fare_class         120000 non-null  int64  
 3   pickup_longitude   120000 non-null  float64
 4   pickup_latitude    120000 non-null  float64
 5   dropoff_longitude  120000 non-null  float64
 6   dropoff_latitude   120000 non-null  float64
 7   passenger_count    120000 non-null  int64  
 8   dist_km            120000 non-null  float64
dtypes: float64(6), int64(2), object(1)
memory usage: 8.2+ MB
None


In [5]:
# Engineering datetime object(string) to be DateTime object.
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   pickup_datetime    120000 non-null  datetime64[ns, UTC]
 1   fare_amount        120000 non-null  float64            
 2   fare_class         120000 non-null  int64              
 3   pickup_longitude   120000 non-null  float64            
 4   pickup_latitude    120000 non-null  float64            
 5   dropoff_longitude  120000 non-null  float64            
 6   dropoff_latitude   120000 non-null  float64            
 7   passenger_count    120000 non-null  int64              
 8   dist_km            120000 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(6), int64(2)
memory usage: 8.2 MB
None
            pickup_datetime  fare_amount  fare_class  pickup_longitude  \
0 2010-04-19 08:17:56+00:00          6.5           0        -73.99

In [6]:
my_time = df['pickup_datetime'][0]
print(my_time)
print(my_time.hour)

2010-04-19 08:17:56+00:00
8


In [7]:
df['EDTDate'] = df['pickup_datetime'].dt.tz_convert('US/Eastern')
df['Hour'] = df['EDTDate'].dt.hour
df['AMorPM'] = np.where(df['Hour'] >= 12, 'pm', 'am')
df['Weekday'] = df['EDTDate'].dt.strftime('%a')

print(df.head())

            pickup_datetime  fare_amount  fare_class  pickup_longitude  \
0 2010-04-19 08:17:56+00:00          6.5           0        -73.992365   
1 2010-04-17 15:43:53+00:00          6.9           0        -73.990078   
2 2010-04-17 11:23:26+00:00         10.1           1        -73.994149   
3 2010-04-11 21:25:03+00:00          8.9           0        -73.990485   
4 2010-04-17 02:19:01+00:00         19.7           1        -73.990976   

   pickup_latitude  dropoff_longitude  dropoff_latitude  passenger_count  \
0        40.730521         -73.975499         40.744746                1   
1        40.740558         -73.974232         40.744114                1   
2        40.751118         -73.960064         40.766235                2   
3        40.756422         -73.971205         40.748192                1   
4        40.734202         -73.905956         40.743115                1   

    dist_km                   EDTDate  Hour AMorPM Weekday  
0  2.126312 2010-04-19 04:17:56-04:00

In [8]:
print("""
# #################################################################################################
#  Full ANN Code Along - Regression Part Two(2) (Categorical and Continuous Features)
# #################################################################################################
""")


# #################################################################################################
#  Full ANN Code Along - Regression Part Two(2) (Categorical and Continuous Features)
# #################################################################################################



In [9]:
categorical_cols = ['Hour', 'AMorPM', 'Weekday']
continuous_cols = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
                   'dropoff_latitude', 'passenger_count', 'dist_km']
y_col = ['fare_amount']     # Hence regression problem.

In [10]:
# Change the categorical columns's data type('dtype') to 'category' type, so neural network can understand
# assigning a number.
for col in categorical_cols:
    df[col] = df[col].astype('category')

In [11]:
df['Hour'].head()

0     4
1    11
2     7
3    17
4    22
Name: Hour, dtype: category
Categories (24, int64): [0, 1, 2, 3, ..., 20, 21, 22, 23]

In [12]:
df['AMorPM'].head()

0    am
1    am
2    am
3    pm
4    pm
Name: AMorPM, dtype: category
Categories (2, object): [am, pm]

In [13]:
df['Weekday'].head()

0    Mon
1    Sat
2    Sat
3    Sun
4    Fri
Name: Weekday, dtype: category
Categories (7, object): [Fri, Mon, Sat, Sun, Thu, Tue, Wed]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype                     
---  ------             --------------   -----                     
 0   pickup_datetime    120000 non-null  datetime64[ns, UTC]       
 1   fare_amount        120000 non-null  float64                   
 2   fare_class         120000 non-null  int64                     
 3   pickup_longitude   120000 non-null  float64                   
 4   pickup_latitude    120000 non-null  float64                   
 5   dropoff_longitude  120000 non-null  float64                   
 6   dropoff_latitude   120000 non-null  float64                   
 7   passenger_count    120000 non-null  int64                     
 8   dist_km            120000 non-null  float64                   
 9   EDTDate            120000 non-null  datetime64[ns, US/Eastern]
 10  Hour               120000 non-null  category                  
 11  

In [15]:
# We can access 'category' type object details using '.cat' as in '.dt' for 'DateTime' objects.
print(df['AMorPM'].cat.categories)
print(df['AMorPM'].cat.codes)

vals = df['AMorPM'].cat.codes.values
print(type(vals))
print(vals)

Index(['am', 'pm'], dtype='object')
0         0
1         0
2         0
3         1
4         1
         ..
119995    0
119996    0
119997    1
119998    0
119999    1
Length: 120000, dtype: int8
<class 'numpy.ndarray'>
[0 0 0 ... 1 0 1]


In [16]:
hr = df['Hour'].cat.codes.values
ampm = df['AMorPM'].cat.codes.values
wkd = df['Weekday'].cat.codes.values

cat = np.stack([hr, ampm, wkd], axis=1) # Join each array as a column thus, axis=1.
print(cat)

[[ 4  0  1]
 [11  0  2]
 [ 7  0  2]
 ...
 [14  1  3]
 [ 4  0  5]
 [12  1  2]]


In [17]:
# Use 'list' comprehension to derive values and stack them up to single numpy array.
cat = np.stack([df[col].cat.codes.values for col in categorical_cols], axis=1)
print(cat)

# OR skip change data type 'for loop' and embed it into the list comprehension as well.

cat = np.stack([df[col].astype('category').cat.codes.values for col in categorical_cols], axis=1)
print(cat)

[[ 4  0  1]
 [11  0  2]
 [ 7  0  2]
 ...
 [14  1  3]
 [ 4  0  5]
 [12  1  2]]
[[ 4  0  1]
 [11  0  2]
 [ 7  0  2]
 ...
 [14  1  3]
 [ 4  0  5]
 [12  1  2]]


In [18]:
# Convert numpy array to 'tensor'.
cat = torch.tensor(cat, dtype=torch.int64)
print(cat)

tensor([[ 4,  0,  1],
        [11,  0,  2],
        [ 7,  0,  2],
        ...,
        [14,  1,  3],
        [ 4,  0,  5],
        [12,  1,  2]])


In [19]:
# Continuous columns - Simply map them into a numpy since they are already numerical values and neurons basicaly
# understand them.

cont = np.stack([df[col].values for col in continuous_cols], axis=1)
print(cont)

# To a 'tensor'.
cont = torch.tensor(cont, dtype=torch.float)
print(cont)

[[-73.992365    40.730521   -73.975499    40.744746     1.
    2.12631159]
 [-73.990078    40.740558   -73.974232    40.744114     1.
    1.39230687]
 [-73.994149    40.751118   -73.960064    40.766235     2.
    3.32676344]
 ...
 [-73.988574    40.749772   -74.011541    40.707799     3.
    5.05252282]
 [-74.004449    40.724529   -73.992697    40.730765     1.
    1.20892296]
 [-73.955415    40.77192    -73.967623    40.763015     3.
    1.42739869]]
tensor([[-73.9924,  40.7305, -73.9755,  40.7447,   1.0000,   2.1263],
        [-73.9901,  40.7406, -73.9742,  40.7441,   1.0000,   1.3923],
        [-73.9941,  40.7511, -73.9601,  40.7662,   2.0000,   3.3268],
        ...,
        [-73.9886,  40.7498, -74.0115,  40.7078,   3.0000,   5.0525],
        [-74.0044,  40.7245, -73.9927,  40.7308,   1.0000,   1.2089],
        [-73.9554,  40.7719, -73.9676,  40.7630,   3.0000,   1.4274]])


In [20]:
# CREATE the labels using 'fare_amount' columns, hence need to predict 'fare_amount' base on categorical and continuous
# columns values after training.
y = torch.tensor(df['fare_amount'].values, dtype=torch.float32).reshape(-1, 1)
print(y)

tensor([[ 6.5000],
        [ 6.9000],
        [10.1000],
        ...,
        [12.5000],
        [ 4.9000],
        [ 5.3000]])


In [21]:
# After all the data prepared.
print(cat.shape)
print(cont.shape)
print(y.shape)

torch.Size([120000, 3])
torch.Size([120000, 6])
torch.Size([120000, 1])


In [22]:
# #################################################################################################
# SET EMBEDDING Sizes
# #################################################################################################
# 1. Step - Take category sizes.
cat_szs = [len(df[col].cat.categories) for col in categorical_cols]
print(cat_szs)

# 2. Step - Take embedded sizes.
embedding_szs = [(szs, min(50, (szs + 1) // 2)) for szs in cat_szs]
print(embedding_szs)

[24, 2, 7]
[(24, 12), (2, 1), (7, 4)]


In [23]:
print("""
# ##############################################################################################################
#  Full ANN Code Along - Regression Part Three(3) (Tabular Model - Embedding, Normalization, Dropout functions)
# ##############################################################################################################
""")


# ##############################################################################################################
#  Full ANN Code Along - Regression Part Three(3) (Tabular Model - Embedding, Normalization, Dropout functions)
# ##############################################################################################################



In [24]:
# Embedding for categorical data.
cats_part = cat[:4]
print(cats_part)

self_embeds = nn.ModuleList([nn.Embedding(vocab_szs, vector_szs) for vocab_szs, vector_szs in embedding_szs])
print(self_embeds)

tensor([[ 4,  0,  1],
        [11,  0,  2],
        [ 7,  0,  2],
        [17,  1,  3]])
ModuleList(
  (0): Embedding(24, 12)
  (1): Embedding(2, 1)
  (2): Embedding(7, 4)
)


In [37]:
# FORWARD METHOD (cats)
embeddings = []
for i, e in enumerate(self_embeds):
    embeddings.append(e(cats_part[:, i]))

print(len(embeddings[0]))

4


In [41]:
# Concatenate embeddings together.
z = torch.cat(embeddings, 1)
print(z.shape)


torch.Size([4, 17])


In [27]:
# Dropout some values from dense-vector to avoid over-fitting.
self_embed_dropout = nn.Dropout(p=0.4)
z = self_embed_dropout(z)

print(z)

tensor([[ 2.7924, -1.5316,  0.5040,  0.0000, -1.8812, -0.0000,  0.0000,  0.0000,
         -0.0000,  0.6651, -1.7901, -0.0000, -0.0000, -0.0000,  0.4195, -3.9756,
         -0.0000],
        [ 1.1120,  2.3520,  1.9108,  1.9899, -1.2272,  0.0000, -0.3299, -2.5737,
          0.1654, -0.9220,  2.0288, -0.0569, -0.1717,  1.3861, -0.7307,  0.0000,
         -0.3266],
        [ 0.0000,  0.2085,  1.9706,  0.0000,  0.8714, -0.1165, -0.9618,  2.9078,
          1.2018, -0.3691, -1.8237,  2.6186, -0.0000,  1.3861, -0.7307,  0.0000,
         -0.0000],
        [ 2.9508,  0.0000, -1.9123,  1.6128, -1.5949,  1.3373,  0.0000,  0.5136,
          3.4139, -2.6294, -0.0000, -2.5356,  0.0000,  0.0000, -1.1570, -0.0000,
          2.3726]], grad_fn=<MulBackward0>)


In [28]:
# Build the 'TabularModel' class.
class TabularModel(nn.Module):

    def __init__(self, embedding_size, num_continuous_f, output_size, layers, p=0.5):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(vocab_szs, vector_szs) for vocab_szs, vector_szs in embedding_size])
        self.batch_norm = nn.BatchNorm1d(num_continuous_f)
        self.dropout = nn.Dropout(p)

        num_embeds = sum([vector_szs for vocab_szs, vector_szs in embedding_size])
        input_size = num_embeds + num_continuous_f

        layer_list = []
        for l in layers:
            layer_list.append(nn.Linear(input_size, l))
            layer_list.append(nn.ReLU(inplace=True))
            layer_list.append(nn.BatchNorm1d(l))
            layer_list.append(nn.Dropout(p))
            input_size = l

        layer_list.append(nn.Linear(layers[-1], output_size))

        self.layer = nn.Sequential(*layer_list)

    def forward(self, x_cat, x_cont):
        _embeddings = []
        for n, emb in enumerate(self.embeddings):
            _embeddings.append(emb(x_cat[:, n]))

        x = torch.cat(_embeddings, 1)
        x = self.dropout(x)

        x_cont = self.batch_norm(x_cont)

        x = torch.cat([x, x_cont], 1)

        return self.layer(x)

In [29]:
# What does 'nn.Sequential' do.
s = nn.Sequential(nn.Linear(10, 20), nn.Linear(20, 10))
print(s)

input = torch.linspace(0, 10, 10)
output = s(input)
print(output)

Sequential(
  (0): Linear(in_features=10, out_features=20, bias=True)
  (1): Linear(in_features=20, out_features=10, bias=True)
)
tensor([ 0.1831,  0.7487, -3.3607,  0.5072,  0.3675,  0.7700,  0.3308, -2.0476,
         2.0286, -0.5559], grad_fn=<AddBackward0>)


In [30]:
torch.manual_seed(33)

# embed_size = [(size, min(50, (size + 1) // 2)) for size in [len(df[col].cat.categories) for col in categorical_cols]]
print(embedding_szs)

model = TabularModel(embedding_szs, cont.shape[1], 1, [200, 100], p=0.4)
print(model)


[(24, 12), (2, 1), (7, 4)]
TabularModel(
  (embeddings): ModuleList(
    (0): Embedding(24, 12)
    (1): Embedding(2, 1)
    (2): Embedding(7, 4)
  )
  (batch_norm): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (layer): Sequential(
    (0): Linear(in_features=23, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=100, out_features=1, bias=True)
  )
)


In [31]:
criterion = nn.MSELoss()    # np.sqrt(MSE) -> Root Mean Squared Error
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

batch_size = 60000
test_size = int(batch_size * 0.2)

# DATA SHUFFLED ALREADY.
cat_train = cat[:batch_size - test_size]
cat_test = cat[batch_size - test_size:batch_size]
cont_train = cont[:batch_size - test_size]
cont_test = cont[batch_size - test_size:batch_size]

y_train = y[:batch_size - test_size]
y_test = y[batch_size - test_size:batch_size]

print(len(cat_train))
print(len(cont_train))
print(len(y_train))
print(len(cat_test))
print(len(cont_test))
print(len(y_test))

48000
48000
48000
12000
12000
12000


In [32]:
start_time = time.time()

epochs = 300
losses = []

for i in range(epochs):

    y_pred = model.forward(cat_train, cont_train)
    # Calculate Root Mean Square Error hence dealing with price units, otherwise loss values
    # (in this case 'fare_amount') would be squared.
    loss = torch.sqrt(criterion(y_pred, y_train))  # Hence the dealing with prices.

    losses.append(loss)

    if i % 10 == 1: # Print the very first one as well.
        print(f'\repoch {i} -> loss: {loss}')

    # Back propagate.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

duration = time.time() - start_time # In seconds.
print(f'Training took {duration / 60} minutes')

epoch 1 -> loss: 11.779982566833496
epoch 11 -> loss: 9.593120574951172


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'full_ann_state.pt')

In [ ]:
plt.plot(range(epochs), losses)
plt.xlabel('Epochs')
plt.ylabel('RMSE Loss')

In [ ]:
# 1. Check the loss immediate loss values.
with torch.no_grad():

    y_pred = model.forward(cat_test, cont_test)

    loss = torch.sqrt(criterion(y_pred, y_test))

    print(loss) # tensor(3.1052) - pretty much good value. No over-fitting.

# 2. Check with the entire test result set.
for i in range(10):
    diff = np.abs(y_pred[i].item() - y_test[i].item())
    print(f'{i}.) PREDICTED: {y_pred[i].item():8.2f}, ACTUAL: {y_test[i].item():8.2f}, DIFF: {diff} ')